# y. Configuration

RADICAL-Pilot (RP) uses a resource configuration process to set control and management parameters for the initialization of its components and to define resource entry points.

It includes:
- Resource description
   - Batch system
   - Provided launch methods (e.g., `srun`, `mpirun`, etc.)
   - Environment setup (including package manager, working directory, etc.)
   - Entry points: batch system URL, file system URL
- Run description
   - Project allocation name (i.e., `account`/`project`)
   - Job queue name on a remote machine
   - Amount of the resources for the run (e.g., `cores`, `gpus`, `memory`) for a particular period of time `runtime`

All this information is provided for the `Pilot` by a `PilotDescription` object.

## y.1. Resource description

Resource description (i.e., resource configuration) is represented by a record of a particular resource within a JSON file, which collects resources per facility and is named as `resource_<facility_name>.json`. Resource description label is set as `<facility_name>.<resource_name>` and is referenced in `PilotDescription` (e.g., `pd.resource="ornl.summit"`)

- Storage location for a pre-defined resource configurations ([GitHub repo](https://github.com/radical-cybertools/radical.pilot/tree/master/src/radical/pilot/configs))
   - `<ve>/lib/python<py_ver>/site-packages/radical/pilot/configs/`
- Storage location for user-defined resource configurations
   - `$HOME/.radical/pilot/configs/` (pre-set `$RADICAL_CONFIG_USER_DIR` will reset a base directory for user configs)



## y.2. Run description

### y.2.1. Resource allocation parameters

### y.2.2. Resource access schema

Resource access schema (`access_schema`) - provided as part of the resource description, and in case of more than one access schemas user can set a specific one in `PilotDescription`
   - `local` - run RP application on the resource machine
   - `ssh` - run RP application outside of the resource machine and use `ssh` protocol and corresponding SSH client to access the resource remotely
   - `gsissh` - run RP application outside of the resource machine and use GSI-enabled SSH to access the resource remotely
      - It adds support for GSI authentication and credential forwarding, providing a single sign-on remote login and file transfer service (for details please follow the [link](http://grid.ncsa.illinois.edu/ssh/))
      - For example, XSEDE resources require using `gsissh` instead of `ssh`. Once the gsissh and myproxy are successfully installed, one need to acquire a X509 certificate
      - ```bash
   
      export MYPROXY_SERVER_PORT=7512
      export MYPROXY_SERVER=myproxy.xsede.org
      myproxy-logon -l <user_name> -T -b -t 1000
      [Enter MyProxy pass phrase]
      [you should receive a credential in /tmp/x509up_u1000]
      gsissh -p 2222 login1.stampede2.tacc.utexas.edu
      ```

**...TBD...**